In [43]:
!pip install -q pyinterval
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [44]:
def moore(f, df, start, end, e, tbl):
    x = interval([start, end])
    width = calculate_width(x)
    Fx = f(x)

    # If no 0 in F(X) -> no root
    if 0 not in Fx:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return
        
    # If interval width less than epsilon -> root
    if width < e:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, 'Root']
        return
        
    # Half if 0 in F'(X)
    x_middle = calculate_midpoint(x)
    dFx = df(x)
    if 0 in dFx:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        moore(f, df, start, x_middle, e, tbl)
        moore(f, df, x_middle, end, e, tbl)
        return
        
    # Stop if x_i+1 is empty
    f_middle = f(x_middle)
    U = x_middle - f_middle / dFx
    x_next = U & x
    if not x_next:
        tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
        return

    # Continue with narrowed interval
    tbl.loc[len(tbl)] = [interval_to_str(x), interval_to_str(Fx), width, '']
    moore(f, df, x_next[0].inf, x_next[0].sup, e, tbl)

In [45]:
func = lambda x: imath.sin(x-1)*(x-2)*(x-3)
dfunc = lambda x: (x**2-5*x+6)*imath.cos(x-1)-(5-2*x)*imath.sin(x-1)
e = 1e-6
a = 0.9
b = 2.3

In [46]:
moore_result = pd.DataFrame(columns=['Interval', 'Interval extension', 'Width', 'Root'])
moore(func, dfunc, a, b, e, moore_result)
moore_result

,Interval,Interval extension,Width,Root
0,"[0.9000000, 2.3000000]","[-0.6070417, 2.2258194]",1.400000e+00,
1,"[0.9000000, 1.6000000]","[-0.2306152, 1.3043241]",7.000000e-01,
2,"[0.9000000, 1.2500000]","[-0.2306152, 0.5715031]",3.500000e-01,
3,"[0.9000000, 1.0750000]","[-0.2306152, 0.1730876]",1.750000e-01,
4,"[0.9960611, 1.0089384]","[-0.0079244, 0.0179824]",1.287733e-02,
5,"[0.9999063, 1.0000695]","[-0.0001875, 0.0001390]",1.632616e-04,
6,"[1.0000000, 1.0000000]","[-0.0000000, 0.0000000]",9.889773e-09,Root
7,"[1.0750000, 1.2500000]","[0.0983452, 0.4405337]",1.750000e-01,
8,"[1.2500000, 1.6000000]","[0.1385462, 0.7410932]",3.500000e-01,
9,"[1.6000000, 2.3000000]","[-0.4046944, 0.5395926]",7.000000e-01,
